In [4]:
position = -6

import pandas as pd
import numpy as np

pm=open('ppm_98_TMRstart.fas','r')
npm=open('npm_98_TMRstart.fas','r')

fas = [[]]

while True:
    reading = pm.readline()
    reading = reading.rstrip()

    if not reading.startswith('>'):
        if reading == '':
            break
        fas.append(list(reading))

fas.remove([])
columns=list(range(-25,26))
df_pm=pd.DataFrame(fas,columns=columns)

fas = [[]]

while True:
    reading = npm.readline()
    reading = reading.rstrip()

    if not reading.startswith('>'):
        if reading == '':
            break
        fas.append(list(reading))

fas.remove([])
columns=list(range(-25,26))
df_npm=pd.DataFrame(fas,columns=columns)


df_pm_freq=df_pm[position].value_counts(sort=True)
df_pm_nor=df_pm[position].value_counts(sort=True,normalize=True)

df_npm_freq=df_npm[position].value_counts(sort=True)
df_npm_nor=df_npm[position].value_counts(sort=True,normalize=True)

printout=pd.concat([df_pm_freq,df_pm_nor,df_npm_freq,df_npm_nor],axis='columns')

printout=printout.set_axis(['pm','pm_normalized','npm','npm_normalized'],axis='columns')

index1=printout.index

printout=printout.assign(AA=index1)

l_order = {'X': 0, 'L': 1, 'I': 2, 'V': 3,'F': 4, 'P': 5, 'M': 6, 'G': 7, 'A': 8, 'T': 9, 
         'W': 10,'K': 11,'H': 12,'R':13,'D': 14,'E':15, 'Q': 16,'N':17,'C': 18,'S':19,'Y': 20}

printout['order']=printout['AA'].map(l_order)

printout=printout.sort_values('order')

for i in range(20):
    if i != printout.iat[i,5]:
        
        for k,v in l_order.items():
            if i==v:
                kcap=k
        
        s=pd.DataFrame({'pm':[0],'pm_normalized':[0],
                        'npm':[0],'npm_normalized':[0],
                        'AA':kcap,'order':[i]},
                       index=[kcap])
        printout=printout.append(s)
        printout=printout.sort_values('order')

printout=printout.fillna(0)

printout['pm']=printout['pm'].astype('int')

# X抜きでnormalizedを出力

else_out=printout.drop('X')

sump=else_out['pm'].sum()
sumn=else_out['npm'].sum()

for i in range (19):
    nump=else_out.iat[i,0]
    numn=else_out.iat[i,2]
    
    else_out.iat[i,1]=100*nump/sump
    else_out.iat[i,3]=100*numn/sumn
    printout.iat[i,1]=100*printout.iat[i,1]
    printout.iat[i,3]=100*printout.iat[i,3]
    
    
else_out=else_out[['pm_normalized','npm_normalized']]

else_out=else_out.rename(columns={'pm_normalized': 'pm_Xout', 'npm_normalized': 'npm_Xout'})

printout=pd.concat([printout,else_out],axis='columns')

printout=printout.reindex(columns=['order', 'pm', 'npm', 'pm_normalized', 'npm_normalized',  'pm_Xout', 'npm_Xout'])

printout

,order,pm,npm,pm_normalized,npm_normalized,pm_Xout,npm_Xout
X,0,7,51,8.974359,9.058615,NaN,NaN
L,1,6,69,7.692308,12.255773,8.450704,13.476562
I,2,1,17,1.282051,3.019538,1.408451,3.320312
V,3,0,32,0.000000,5.683837,0.000000,6.250000
F,4,3,19,3.846154,3.374778,4.225352,3.710938
P,5,9,34,11.538462,6.039076,12.676056,6.640625
M,6,2,13,2.564103,2.309059,2.816901,2.539062
G,7,7,27,8.974359,4.795737,9.859155,5.273438
A,8,3,36,3.846154,6.394316,4.225352,7.031250
T,9,3,21,3.846154,3.730018,4.225352,4.101562
